In [ ]:
import numpy as np
from scipy.spatial import KDTree
from astropy.io import fits
from astropy.table import QTable
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import Planck18
import pylab as plt

In [ ]:
rng = np.random.default_rng(17)

In [ ]:
datafd = fits.open("../data/gaia_G20.0.fits") # note terrible file path
data_table = QTable(datafd[1].data)
print(len(data_table))

In [ ]:
randfd = fits.open("../data/random_stardustm1064_G20.0_10x.fits") # note terrible file path
rand_table = QTable(randfd[1].data)
print(len(rand_table))

In [ ]:
# cut everything by the dust map
ebvlim = 0.05
Id = data_table['ebv'] < ebvlim
data_table = data_table[Id]
Ir = rand_table['ebv'] < ebvlim
rand_table = rand_table[Ir]

In [ ]:
# Add l, b to the random catalog.
# WARNING: This might be inconsistent with how l, b are calculated in the data table.
randc = SkyCoord(ra=rand_table['ra']*u.degree, dec=rand_table['dec']*u.degree, frame='icrs')
rand_table['l'] = randc.galactic.l.to(u.deg).value
rand_table['b'] = randc.galactic.b.to(u.deg).value

In [ ]:
# make two hemsisphere samples
dhi, dlo = data_table['b'] > 0., data_table['b'] < 0.
rhi, rlo = rand_table['b'] > 0., rand_table['b'] < 0.

In [ ]:
for I in [dhi, dlo]:
    plt.scatter(data_table['ra'][I], data_table['dec'][I],
                marker=".", s=0.1, alpha=0.1)
plt.figure()
for I in [rhi, rlo]:
    plt.scatter(rand_table['ra'][I], rand_table['dec'][I],
                marker=".", s=0.1, alpha=0.1)

In [ ]:
for I in [dhi, dlo]:
    plt.scatter(data_table['l'][I], np.sin(data_table['b'][I] * np.pi / 180.),
                marker=".", s=0.1, alpha=0.1)
plt.figure()
for I in [rhi, rlo]:
    plt.scatter(rand_table['l'][I], np.sin(rand_table['b'][I] * np.pi / 180.),
                marker=".", s=0.1, alpha=0.1)

In [ ]:
factor = len(data_table) / len(rand_table)

In [ ]:
bins = np.arange(0., 4.501, 0.1)
bincntrs = 0.5 * (bins[1:] + bins[:-1])
binwids = (bins[1:] - bins[:-1])
nzhi, _ = np.histogram(data_table['redshift_spz'][dhi], bins)
nzhi = nzhi.astype(float) / (np.sum(rhi) * factor) / binwids
nzlo, _ = np.histogram(data_table['redshift_spz'][dlo], bins)
nzlo = nzlo.astype(float) / (np.sum(rlo) * factor) / binwids

In [ ]:
plt.plot(bincntrs, nzhi, "o")
plt.plot(bincntrs, nzlo, "o")

In [ ]:
# give the random points redshifts
rand_table['redshift_spz'] = rng.choice(data_table['redshift_spz'],
                                        size=len(rand_table), replace=True)

In [ ]:
data_table['comoving_distance'] = Planck18.comoving_distance(data_table['redshift_spz']).value
rand_table['comoving_distance'] = Planck18.comoving_distance(rand_table['redshift_spz']).value

In [ ]:
def add_xyz(table):
    r = table['comoving_distance']
    cb = np.cos(table['b'] * np.pi / 180.)
    x = r * cb * np.cos(table['l'] * np.pi / 180.)
    y = r * cb * np.sin(table['l'] * np.pi / 180.)
    z = r * np.sin(table['b'] * np.pi / 180.)
    table['xyz'] = np.array([x, y, z]).T
    return

add_xyz(data_table)
add_xyz(rand_table)

In [ ]:
for I in [dhi, dlo]:
    plt.scatter(data_table['xyz'][I,0], data_table['xyz'][I,2],
                marker=".", s=0.1, alpha=0.1)
plt.figure()
for I in [dhi, dlo]:
    plt.scatter(data_table['xyz'][I,1], data_table['xyz'][I,2],
                marker=".", s=0.1, alpha=0.1)

In [ ]:
data_tree = KDTree(data_table['xyz'])
dhi_tree = KDTree(data_table['xyz'][dhi])
dlo_tree = KDTree(data_table['xyz'][dlo])
rand_tree = KDTree(rand_table['xyz'])
rhi_tree = KDTree(rand_table['xyz'][rhi])
rlo_tree = KDTree(rand_table['xyz'][rlo])

In [ ]:
rs = np.exp(np.arange(np.log(10.), np.log(500.), 0.25))
DD_hi = dhi_tree.count_neighbors(data_tree, rs).astype(float)
DD_lo = dlo_tree.count_neighbors(data_tree, rs).astype(float)

In [ ]:
# Make DR counts (symmetrically)
DR_hi = 0.5 * factor * (dhi_tree.count_neighbors(rand_tree, rs)
                      + rhi_tree.count_neighbors(data_tree, rs)).astype(float)
DR_lo = 0.5 * factor * (dlo_tree.count_neighbors(rand_tree, rs)
                      + rlo_tree.count_neighbors(data_tree, rs)).astype(float)

In [ ]:
# make RR counts
RR_hi = factor ** 2 * rhi_tree.count_neighbors(rand_tree, rs).astype(float)
RR_lo = factor ** 2 * rlo_tree.count_neighbors(rand_tree, rs).astype(float)

In [ ]:
plt.plot(rs, DD_hi, "o", label="DD counts, NGC")
plt.plot(rs, DR_hi, "o", color="blue", alpha=0.25, label="DR counts, NGC")
plt.plot(rs, RR_hi, "o", color="blue", alpha=0.1, label="RR counts, NGC")
plt.plot(rs, DD_lo, "o", label="DD counts, SGC")
plt.plot(rs, DR_lo, "o", color="orange", alpha=0.25, label="DR counts, SGC")
plt.plot(rs, RR_lo, "o", color="orange", alpha=0.1, label="RR counts, SGC")
plt.loglog()
plt.legend()

In [ ]:
LS_hi = (DD_hi - 2 * DR_hi + RR_hi) / RR_hi
LS_lo = (DD_lo - 2 * DR_lo + RR_lo) / RR_lo

In [ ]:
plt.plot(rs, LS_hi, "o", label="LS integral, NGC")
plt.plot(rs, LS_lo, "o", label="LS integral, SGC")
plt.loglog()
plt.legend()